# Benodigde libraries importeren

In [92]:
import pandas as pd
from datetime import datetime
import numpy as np
import pandas.api.types as ptypes


# Bestand inladen en data filteren

In [93]:
# excel bestand maxilia geimporteerd en kolommen gefilterd 

df = pd.read_csv('expdata 20190912 1002.csv', sep = None, delimiter = None, usecols = ['email','Orderdate','OrderID','TurnoverLead', 'BuyinLead', 'LeadPhase','LeadStatus'], engine = 'python') 
df.head()

,OrderID,Orderdate,email,TurnoverLead,BuyinLead,LeadPhase,LeadStatus
0,27654,22-4-2014,janneke.van.wingerden@hotmail.com,840.0,0.0,Order,Gefactureerd
1,30777,9-4-2015,marloessanders@izit.nl,300.0,0.0,Order,Gefactureerd
2,32291,30-7-2014,info@ferox-design.com,235.0,0.0,Order,Gefactureerd
3,32291,30-7-2014,info@ferox-design.com,235.0,0.0,Order,Gefactureerd
4,33824,27-5-2016,johan@thermservice.be,0.0,0.0,Order,Gefactureerd


In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 273322 entries, 0 to 273321
Data columns (total 7 columns):
OrderID         273322 non-null int64
Orderdate       131183 non-null object
email           272921 non-null object
TurnoverLead    273322 non-null float64
BuyinLead       273322 non-null float64
LeadPhase       273322 non-null object
LeadStatus      271268 non-null object
dtypes: float64(2), int64(1), object(4)
memory usage: 14.6+ MB


In [95]:
# lege rijen 'NaT' eruit gefilterd 
# Email bevat @ controle
# Duplicaten verwijderen

df = df.dropna(axis=0, how = 'any', inplace=False)
df = df[~df['email'].str.contains('@')==False]
df = df.drop_duplicates('OrderID', keep='first')
df.head()

,OrderID,Orderdate,email,TurnoverLead,BuyinLead,LeadPhase,LeadStatus
0,27654,22-4-2014,janneke.van.wingerden@hotmail.com,840.0,0.0,Order,Gefactureerd
1,30777,9-4-2015,marloessanders@izit.nl,300.0,0.0,Order,Gefactureerd
2,32291,30-7-2014,info@ferox-design.com,235.0,0.0,Order,Gefactureerd
4,33824,27-5-2016,johan@thermservice.be,0.0,0.0,Order,Gefactureerd
5,34406,4-2-2015,miranda@dynadental.com,1185.0,0.0,Order,Gefactureerd


In [96]:
df['Orderdate']=pd.to_datetime(df['Orderdate'], format='%d-%m-%Y')


In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106583 entries, 0 to 273321
Data columns (total 7 columns):
OrderID         106583 non-null int64
Orderdate       106583 non-null datetime64[ns]
email           106583 non-null object
TurnoverLead    106583 non-null float64
BuyinLead       106583 non-null float64
LeadPhase       106583 non-null object
LeadStatus      106583 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 6.5+ MB


In [98]:
df.describe()

,OrderID,TurnoverLead,BuyinLead
count,106583.000000,106583.000000,106583.00000
mean,147886.233912,611.979496,360.30912
std,61013.462495,1391.539161,1031.11294
min,27654.000000,-392.000000,-1001.51000
25%,94114.000000,167.000000,34.50000
50%,148056.000000,300.400000,151.50000
75%,201786.500000,610.000000,358.47000
max,252808.000000,174519.500000,141338.84000


In [99]:
# Leadphase = order
# Leadstatus = gefactureerd
df = df[~df['LeadPhase'].str.contains('Order')==False]
df = df[~df['LeadStatus'].str.contains('Gefactureerd')==False]
df.head()

,OrderID,Orderdate,email,TurnoverLead,BuyinLead,LeadPhase,LeadStatus
0,27654,2014-04-22,janneke.van.wingerden@hotmail.com,840.0,0.0,Order,Gefactureerd
1,30777,2015-04-09,marloessanders@izit.nl,300.0,0.0,Order,Gefactureerd
2,32291,2014-07-30,info@ferox-design.com,235.0,0.0,Order,Gefactureerd
4,33824,2016-05-27,johan@thermservice.be,0.0,0.0,Order,Gefactureerd
5,34406,2015-02-04,miranda@dynadental.com,1185.0,0.0,Order,Gefactureerd


In [100]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97825 entries, 0 to 272752
Data columns (total 7 columns):
OrderID         97825 non-null int64
Orderdate       97825 non-null datetime64[ns]
email           97825 non-null object
TurnoverLead    97825 non-null float64
BuyinLead       97825 non-null float64
LeadPhase       97825 non-null object
LeadStatus      97825 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 6.0+ MB


In [101]:
# Check data bevat voldoet aan bepaalde condities
assert ptypes.is_datetime64_any_dtype(df['Orderdate'])
assert df['email'].str.contains('@').all()
assert df['LeadPhase'].eq('Order').all()
assert df['LeadStatus'].eq('Gefactureerd').all()


# Data grouperen per klant

In [102]:
# group by customer ID
# hoeveel orders zijn er gedaan per customer
# totale uitgaven per customer 

df_clv = df.groupby('email').agg({
                                                        'Orderdate': lambda date: (date.max() - date.min()).days,
                                                        'OrderID': lambda num: len(num), 
                                                        'TurnoverLead': lambda price: price.sum()
                                                    }) 
df_clv.head()

,Orderdate,OrderID,TurnoverLead
email,,,
03vichy@gmail.com,0,1,259.0
06alexisbrunet@gmail.com,0,1,180.0
070paulus070@gmail.com,0,1,33.5
1.NLD.MINUSMA.GAO.AHDET.COGP.S1@B-SC1.TTN.MINDEF.NL,0,1,1230.5
1.vorsitzende.gv.muenster@gmail.com,0,1,344.5


In [103]:
# rename kolommen 

df_clv.columns = ['days_customer', 'total_transactions', 'total_spent']
df_clv.head()

,days_customer,total_transactions,total_spent
email,,,
03vichy@gmail.com,0,1,259.0
06alexisbrunet@gmail.com,0,1,180.0
070paulus070@gmail.com,0,1,33.5
1.NLD.MINUSMA.GAO.AHDET.COGP.S1@B-SC1.TTN.MINDEF.NL,0,1,1230.5
1.vorsitzende.gv.muenster@gmail.com,0,1,344.5


In [104]:
df_clv.describe()

,days_customer,total_transactions,total_spent
count,67777.000000,67777.000000,67777.000000
mean,76.580684,1.443336,886.926759
std,225.263499,1.724549,2695.224340
min,0.000000,1.000000,-392.000000
25%,0.000000,1.000000,173.000000
50%,0.000000,1.000000,332.000000
75%,0.000000,1.000000,757.600000
max,1686.000000,112.000000,167285.400000


# Berekenen van de totale CLV

In [105]:
# gemiddelde order waarde 

avg_order_value = sum(df_clv['total_spent']) / sum(df_clv['total_transactions'])
print(avg_order_value)

614.4976738052648


In [106]:
# hoe vaak koopt iemand gemiddeld iets

purchase_frequency = sum(df_clv['total_transactions']) / df_clv.shape[0]
print(purchase_frequency)

1.4433362350059755


In [107]:
# hoe vaak komt iemand terug

repeat_rate = df_clv[df_clv.total_transactions > 1].shape[0] / df_clv.shape[0] 
print(repeat_rate)

0.20118919397435708


In [108]:
# churnrate: hoeveel % doet na 1 aankoop geen bestelling meer

churn_rate = 1 - repeat_rate
print(churn_rate)

0.7988108060256429


In [109]:
# gemiddelde marge  

profit = sum(df['TurnoverLead']) - sum(df['BuyinLead'])
margin = profit / sum(df['TurnoverLead'])
    
print(margin)

0.4098393753820391


In [110]:
# totale clv & clv marge 
# in het blok hierboven is de marge berekend ('margin'), maar je kan ook een getal noteren ipv 'margin'

total_clv = (avg_order_value * purchase_frequency) / churn_rate 
total_clv_margin = total_clv * margin

print(total_clv) # het eerste getal is de totale clv
print(total_clv_margin) # het tweede getal is de clv waarbij rekening is gehouden met de berekende marge

1110.3089145010256
455.04831200021016


In [111]:
# Bovenstaande cijfers in kolom neerzetten zodat de data in datastudio kan worden gebruikt
set_total_clv = avg_order_value, purchase_frequency, churn_rate, total_clv, total_clv_margin 
df_total_clv = pd.DataFrame(set_total_clv, columns= ['Waarde'], index = ['Gemiddelde orderwaarde' , 'Frequentie aankoop', 'Churn rate' , 'CLV', 'CLV marge'])
df_clv_values=df_total_clv.T.groupby(level=0).agg(lambda x : x.values.tolist()).stack().apply(pd.Series).unstack().sort_index(level=1,axis=1)
df_clv_values.columns=df_clv_values.columns.droplevel(level=0)
df_clv_values.head()

,CLV,CLV marge,Churn rate,Frequentie aankoop,Gemiddelde orderwaarde
Waarde,1110.308915,455.048312,0.798811,1.443336,614.497674


In [112]:
df_clv_values.to_excel('CLV_waardes.xlsx')

   # Transacties en churn rate per maand uiteenzetten

In [113]:
# transacties per klant per maand uiteenzetten
df_monthly = df.set_index('Orderdate')
df_monthly = df_monthly.pivot_table(
    index=['email'],
    columns=pd.Grouper(freq='M'),
    values='OrderID', 
    aggfunc='count',
    fill_value=0
)

# Format column headers to human readable format
#df_monthly.columns = [x.strftime('%b %Y') for x in df_monthly.columns]
df_monthly.head()

Orderdate,2014-04-30 00:00:00,2014-07-31 00:00:00,2014-09-30 00:00:00,2014-10-31 00:00:00,2014-11-30 00:00:00,2014-12-31 00:00:00,2015-01-31 00:00:00,2015-02-28 00:00:00,2015-03-31 00:00:00,2015-04-30 00:00:00,...,2018-12-31 00:00:00,2019-01-31 00:00:00,2019-02-28 00:00:00,2019-03-31 00:00:00,2019-04-30 00:00:00,2019-05-31 00:00:00,2019-06-30 00:00:00,2019-07-31 00:00:00,2019-08-31 00:00:00,2019-09-30 00:00:00
email,,,,,,,,,,,,,,,,,,,,,
03vichy@gmail.com,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
06alexisbrunet@gmail.com,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
070paulus070@gmail.com,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1.NLD.MINUSMA.GAO.AHDET.COGP.S1@B-SC1.TTN.MINDEF.NL,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1.vorsitzende.gv.muenster@gmail.com,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [114]:
repeat_rate = df_monthly[df_monthly > 1].count() / df_monthly[df_monthly > 0].count()
df_churn = repeat_rate.to_frame(name='repeat_rate')

In [115]:
df_churn['churn_rate'] = 1 - df_churn['repeat_rate']
df_churn


,repeat_rate,churn_rate
Orderdate,,
2014-04-30,0.000000,1.000000
2014-07-31,0.000000,1.000000
2014-09-30,0.000000,1.000000
2014-10-31,0.000000,1.000000
2014-11-30,0.000000,1.000000
2014-12-31,0.000000,1.000000
2015-01-31,0.034853,0.965147
2015-02-28,0.031627,0.968373
2015-03-31,0.029777,0.970223


# CLV per klant uitrekenen

In [116]:
# clv per customer

df_clv['customer_order_value'] = df_clv['total_spent'] / df_clv['total_transactions']
df_clv['customer_clv'] = (df_clv['customer_order_value'] * purchase_frequency) / churn_rate
df_clv['customer_clv_margin'] = df_clv['customer_clv'] * margin

df_clv.head()

,days_customer,total_transactions,total_spent,customer_order_value,customer_clv,customer_clv_margin
email,,,,,,
03vichy@gmail.com,0,1,259.0,259.0,467.975749,191.794888
06alexisbrunet@gmail.com,0,1,180.0,180.0,325.234111,133.293745
070paulus070@gmail.com,0,1,33.5,33.5,60.529682,24.807447
1.NLD.MINUSMA.GAO.AHDET.COGP.S1@B-SC1.TTN.MINDEF.NL,0,1,1230.5,1230.5,2223.336519,911.210850
1.vorsitzende.gv.muenster@gmail.com,0,1,344.5,344.5,622.461951,255.109417


In [117]:
df_clv.describe()

,days_customer,total_transactions,total_spent,customer_order_value,customer_clv,customer_clv_margin
count,67777.000000,67777.000000,67777.000000,67777.000000,67777.000000,67777.000000
mean,76.580684,1.443336,886.926759,537.520097,971.221505,398.044815
std,225.263499,1.724549,2695.224340,1009.096235,1823.291760,747.256756
min,0.000000,1.000000,-392.000000,-392.000000,-708.287619,-290.284156
25%,0.000000,1.000000,173.000000,162.890000,294.318802,120.623434
50%,0.000000,1.000000,332.000000,290.000000,523.988290,214.751033
75%,0.000000,1.000000,757.600000,567.600000,1025.571563,420.319609
max,1686.000000,112.000000,167285.400000,61600.000000,111302.340186,45616.081580


In [118]:
df_clv.to_excel('CLV_per_klant.xlsx')

In [119]:
# set orderdate als index
df.set_index('Orderdate', inplace=True)

In [120]:
#totale transacties / opbrengst per maand

df_transacties = df.groupby(pd.Grouper(freq='M')).agg(
    {
        'OrderID': len, 
        'TurnoverLead': sum,
        'email': pd.Series.nunique
    }
)

df_transacties.head()

,OrderID,TurnoverLead,email
Orderdate,,,
2014-04-30,1,840.0,1
2014-05-31,0,0.0,0
2014-06-30,0,0.0,0
2014-07-31,1,235.0,1
2014-08-31,0,0.0,0


In [121]:
# Uiteenzetting uitgaves klanten per maand
# df['month_yr'] = df['Orderdate'].apply(lambda x: x.strftime('%b %Y'))
# df.head()


In [122]:
# totale transacties / opbrengst per maand

# df_transacties = df.groupby('month_yr').aggregate({
#                                                        'OrderID': lambda num: len(num), 
#                                                        'TurnoverLead': lambda price: price.sum(),
#                                                        'email': pd.Series.nunique
#                                                    }) 
# df_transacties.head()

In [123]:
df_transacties.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 66 entries, 2014-04-30 to 2019-09-30
Freq: M
Data columns (total 3 columns):
OrderID         66 non-null int64
TurnoverLead    66 non-null float64
email           66 non-null int64
dtypes: float64(1), int64(2)
memory usage: 4.6 KB


In [124]:
df_transacties.columns = ['total_transactions', 'total_spent', 'total_unique_customers']
df_transacties.head(5)

,total_transactions,total_spent,total_unique_customers
Orderdate,,,
2014-04-30,1,840.0,1
2014-05-31,0,0.0,0
2014-06-30,0,0.0,0
2014-07-31,1,235.0,1
2014-08-31,0,0.0,0


In [125]:
df_transacties.describe()

,total_transactions,total_spent,total_unique_customers
count,66.000000,6.600000e+01,66.000000
mean,1482.196970,9.108066e+05,1393.166667
std,922.956741,5.712635e+05,856.845980
min,0.000000,0.000000e+00,0.000000
25%,897.250000,4.805995e+05,871.250000
50%,1395.500000,9.146664e+05,1314.500000
75%,2106.250000,1.291857e+06,1955.750000
max,3810.000000,2.391474e+06,3538.000000


In [126]:
df_transacties.to_excel('Overzicht_per_maand.xlsx')

# CLV per maand berekenen

In [127]:
# CLV per maand berekenen
df_transacties['purchase_frequency_month'] = df_transacties['total_transactions'] / df_transacties['total_unique_customers']
df_transacties['avg_order_value_month'] = df_transacties['total_spent'] / df_transacties['total_transactions'] 
df_transacties['CLV'] = (df_transacties['avg_order_value_month'] * df_transacties['purchase_frequency_month']) / df_churn['churn_rate']
df_transacties['CLV_margin'] = df_transacties['CLV'] * margin
df_transacties.head()

,total_transactions,total_spent,total_unique_customers,purchase_frequency_month,avg_order_value_month,CLV,CLV_margin
Orderdate,,,,,,,
2014-04-30,1,840.0,1,1.0,840.0,840.0,344.265075
2014-05-31,0,0.0,0,NaN,NaN,NaN,NaN
2014-06-30,0,0.0,0,NaN,NaN,NaN,NaN
2014-07-31,1,235.0,1,1.0,235.0,235.0,96.312253
2014-08-31,0,0.0,0,NaN,NaN,NaN,NaN


In [128]:
# New dataframe to calculate CLV averages. Needs "index" & "number of months" helper columns
# Even if some rows ("months") are missing, "number of months" is still correct
# Because it looks at the dates, not the index locations. 
df_clv_cum = df_transacties[['CLV']].copy()
df_clv_cum.dropna(inplace=True)
df_clv_cum['Index'] = np.arange(1, len(df_clv_cum) + 1)
df_clv_cum['CLV_AVG'] = df_clv_cum['CLV'].cumsum().div(df_clv_cum['Index'])
df_clv_cum.head()

,CLV,Index,CLV_AVG
Orderdate,,,
2014-04-30,840.000000,1,840.000000
2014-07-31,235.000000,2,537.500000
2014-09-30,219.000000,3,431.333333
2014-10-31,564.000000,4,464.500000
2014-11-30,675.251111,5,506.650222


In [129]:
# df_clv_cum = pd.DataFrame(df_transacties.iloc[:,-2])
# df_clv_cum['CLV_cumulatief'] = df_clv_cum.rolling(window=4).mean()
# df_clv_cum.head()